In [4]:
#!/usr/bin/env python3
# coding=utf-8
import websocket
import time
import threading
import requests
import json
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import hashlib
import hmac
import math
import sys
import logging
import os
import getopt
import traceback
from scipy import stats
from itertools import product as product
from aligo import Aligo

g_worthDir = '../pkl'  # 
g_parentDir = '../../..'  # 
g_dbfile = f"{g_parentDir}/pkl/hq.db" # 
g_name = g_access = 'logic'

df_balance = pd.DataFrame()
df_posirisk = pd.DataFrame()
df_order = pd.DataFrame()
df_trade = pd.DataFrame()

g_locTimeadj = 28800 # 28800 0
g_version = '1.0.1'

print(f"{sys.version} , {sys.executable} ")
print(f"{time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()+g_locTimeadj))} *****import**v{g_version}*** ")

3.10.4 (main, Nov 29 2022, 20:00:25) [GCC 9.4.0] , /home/codespace/.python/current/bin/python3 
2023-01-02 21:27:07 *****import**v1.0.1*** 


In [6]:
conn = sqlite3.connect('../pkl/binance.db')  # g_dbfile
try:
    df_trade = pd.read_sql(' select * from trade ', conn)
except:
    pass
print(f"{df_trade}")
conn.close()

Empty DataFrame
Columns: []
Index: []


In [ ]:
conn = sqlite3.connect('../pkl/binance.db')
# df_acc0 = pd.read_sql(' select * from account ', conn) 
# rt_df.to_sql('account', con=conn, if_exists='append', index=False) #,
# df_bal0 = pd.read_sql(' select * from balance ', conn) 
df_ord0 = pd.read_sql(' select * from orders ', conn) 
conn.close()

display(df_ord0)

In [ ]:
def savePickle(src_df, dest_df, flag):

    pklFile = f"{os.getcwd()}/origin/{flag}.pkl"

    if dest_df.empty:
        if os.path.exists(pklFile):
            dest_df = pd.read_pickle(pklFile)
        else:
            dest_df = pd.DataFrame()

    dest_df = pd.concat([dest_df, src_df], ignore_index=True)
    dest_df.to_pickle(pklFile)

In [2]:
try:
    from aligo import Aligo
    import time
except:
    # !pip install pandas numpy matplotlib scipy tqdm requests_toolbelt pymysql sqlalchemy flask pandas_datareader  torch  pymssql EMD-signal websocket-client ccxt statsmodels  sklearn

    # os.system('pip install pandas numpy matplotlib scipy tqdm pymysql sqlalchemy ')
    # os.system('pip install requests_toolbelt flask pandas_datareader sshtunnel torch ')
    # os.system('pip install pymssql EMD-signal websocket-client ccxt statsmodels sklearn')
    os.system('pip install --upgrade aligo')
else:
    print( f'{time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time.time()+28800))}')


2022-12-06 20:51:14


In [4]:
from aligo import Aligo

if __name__ == '__main__':
    ali = Aligo(email=('ation126@126.com', 'ation126') )  # 首次扫描二维码 # email=('ation126@126.com', 'ation126')  port=8080 # ali1 = Aligo(name='user1')
    
    user = ali.get_user()  # 获取用户信息
    print(user.user_name, user.nick_name, user.phone)  # 打印用户信息
    
    ll = ali.get_file_list()  # 获取网盘根目录文件列表
    for file in ll:  # 遍历文件列表
        print(file.file_id, file.name, file.type)  # 打印文件信息

12:51:34.742 aligo.INFO Config /home/codespace/.aligo/aligo.json
12:51:34.743 aligo.INFO 日志等级 DEBUG
12:51:35.424 aligo.INFO 类 Unix 操作系统
12:51:35.428 aligo.INFO 加载配置文件 /home/codespace/.aligo/aligo.json
12:51:36.116 aligo.INFO POST https://api.aliyundrive.com/v2/user/get 200 570
12:51:36.306 aligo.INFO POST https://api.aliyundrive.com/adrive/v3/file/list 200 3160


151***426 ation 15157186426
638eae22a6ba5cb7577643af80a42f5b557d42cd backup folder
638dd4cfb94a892c7f794e258ac7d4fdfad3d1a9 Resource folder
60bb49a0430ef482563b41dea47b79e2d889c964 Settings folder
6065997c09a221bf81bb4df0bc31398f3c5f9ad3 tutorial folder
60fb7c95a41823d493574039bfaa7414650c9889 Share folder
6065997545a54ae72e1b4e70980adfd758e5f944 IT folder
60659969cdae7f3424ba4a8499ccdfa09c17bddf Com folder


In [5]:
# 上传文件/夹
remote_folder = ali.get_folder_by_path('/backup/20221206/') # /
ali.upload_folder('/workspaces/jupyter/tmp/', parent_file_id=remote_folder.file_id) # parent_file_id='61021f2e841b7566d37349d2bd5f42ec56885d68'
# ali.update_file(UpdateFileRequest(file_id=test_file, name=new_name))
ali.upload_file('/workspaces/jupyter/tmp/t1.py', parent_file_id=remote_folder.file_id)

12:16:09.957 aligo.INFO POST https://api.aliyundrive.com/adrive/v3/file/list 200 3160
12:16:10.136 aligo.INFO POST https://api.aliyundrive.com/adrive/v3/file/list 200 599
12:16:10.137 aligo.INFO 开始上传文件 /workspaces/jupyter/tmp/t1.py
12:16:10.462 aligo.INFO POST https://api.aliyundrive.com/adrive/v2/file/createWithFolders 409 149
12:16:10.701 aligo.INFO POST https://api.aliyundrive.com/adrive/v2/file/createWithFolders 201 368
12:16:10.702 aligo.INFO 文件秒传成功 /workspaces/jupyter/tmp/t1.py


CreateFileResponse(file_name='t1.py', type='file', file_id='638f328a2efd8502ae1c4605872632a37397ef7f', parent_file_id='638ebc1b51285f016b574feb9177888d6ebe7625')

In [6]:
# 下载文件/夹
file = ali.get_file_by_path('/backup/20221206/t1.py')
local_folder = '/workspaces/jupyter/dev/'
if file.type == 'file':
    ali.download_file(file=file, local_folder=local_folder)
else:
    ali.download_folder(file.file_id, local_folder=local_folder)

12:25:43.681 aligo.INFO POST https://api.aliyundrive.com/adrive/v3/file/list 200 3160
12:25:43.843 aligo.INFO POST https://api.aliyundrive.com/adrive/v3/file/list 200 599
12:25:44.012 aligo.INFO POST https://api.aliyundrive.com/adrive/v3/file/list 200 6509
12:25:44.014 aligo.INFO 开始下载文件 /workspaces/jupyter/dev/t1.py
100%|██████████| 25.3k/25.3k [00:00<00:00, 13.8MB/s]
12:25:44.568 aligo.INFO 文件下载完成 /workspaces/jupyter/dev/t1.py


In [ ]:
# 相册
r = ali.get_albums_info()
print(r.driveId, r.driveName)

In [ ]:
from aligo import Aligo

if __name__ == '__main__':
    ali = Aligo()

    for i in ali.get_file_list(drive_id='6333175'):
        print(i)

    # 获取相册列表
    for album in ali.list_albums():
        """
        6333175 None 3 default  # 注意，云盘中会有一个 默认的 album，无法删除
        6333175 None None starred
        iqbf34LjW2 标题。 3 manual
        k6R1nmadvf6 产品标题。 0 manual
        """
        print(album.album_id, album.name, album.total_count, album.type)

    # 创建相册
    album = ali.create_album('aligo', description='create by aligo')
    print(album.album_id)
    print(album.name)
    print(album.description)
    print(album.owner)
    print(album.created_at)
    print(album.updated_at)
    print(album.file_count)
    print(album.image_count)
    print(album.video_count)

    # 添加文件到相册
    # 可以从本地或云盘中添加
    # 从本地添加需要先上传
    # f = ali.upload_file(r"C:\Users\foyou\Pictures\Avatar\copilot-logo-large.png")  # 如果在云盘中删除此文件，相册中对应图片也会消失
    f = ali.upload_file(r"C:\Users\foyou\Pictures\Avatar\copilot-logo-large.png", drive_id=ali.album_info.driveId)
    # ali.add_files_to_album(album.album_id, [f])
    ali.add_file_to_album(album.album_id, f)
    # ali.add_file_to_album('LTeF1dwB3Wy', f)

    ll = ali.list_album_files(album.album_id)
    for i in ll:
        print(i)

    # 重命名相册
    r = ali.rename_album(album.album_id, '相册新名字')
    print(r)

    # 删除相册
    # ali.delete_album(album.album_id)


In [ ]:
# 文件（夹）相关  
# 创建文件夹：create_folder  
# 重命名文件（夹）
# rename_file: 重命名文件（夹）
# batch_rename_files: 批量重命名文件（夹）
# 移动文件（夹）
# move_file: 移动文件（夹）
# batch_move_files: 批量移动文件（夹）
# 复制文件（夹）
# copy_file: 复制文件（夹）
# batch_copy_files: 批量复制文件（夹）
# 获取文件（夹）
# get_file: 获取文件（夹）
# batch_get_files: 批量获取文件（夹）
# get_file_by_path: 根据路径获取文件（夹）
# 获取文件列表: get_file_list
# 其他
# get_path: 获取文件（夹）路径信息
# 收藏相关
# starred_file: 收藏和取消收藏 文件（夹）
# batch_star_files: 批量收藏和取消收藏 文件（夹）
# 分享相关
# get_share_list: 获取分享列表
# 分享文件（夹）
# share_file: （批量）分享文件（夹）
# 取消分享文件（夹）
# cancel_share: 取消分享
# batch_cancel_share: 批量取消分享
# update_share: 更新分享
# get_share_info: 获取分享信息
# get_share_token: 获取分享 token
# get_share_file: 获取分享文件（夹）
# get_share_file_list: 获取分享文件列表
# share_file_saveto_drive: 保存分享文件（夹）
# batch_share_file_saveto_drive: 批量保存分享文件（夹）
# get_share_link_download_url:
# 自定义分享
# share_files_by_aligo: 批量分享文件
# share_folder_by_aligo: 分享文件夹
# save_files_by_aligo：保存自定义分享
# 回收站相关
# get_recyclebin_list: 获取回收站文件列表
# 移动文件（夹）至回收站：不提供彻底删除的接口，如需使用，请参考自定义功能
# move_file_to_trash: 移动文件（夹）到回收站
# batch_move_to_trash: 批量移动文件（夹）到回收站
# 从回收站恢复文件（夹）
# restore_file: 恢复回收站文件（夹）
# batch_restore_files: 批量恢复回收站文件（夹）
# 下载相关
# download_file: 下载文件
# download_files: 批量下载文件
# download_folder: 下载文件夹
# 其他
# get_download_url: 获取文件下载地址（一般不直接使用）
# batch_download_url: 批量获取文件下载地址（一般不直接使用）
# 上传相关
# upload_file: 上传文件
# upload_files: 批量上传文件
# upload_folder: 上传文件夹
# 其他
# sync_folder: 文件夹同步

In [1]:
import requests 
url='https://qyapi.weixin.qq.com/cgi-bin/gettoken?'
data = {"corpid":"ww9ca2ccd8f391f581", "corpsecret":"lbjGCu1aNPJafrahotkvOrqrL-m7NfYehPisHMnthro"}

res = requests.get(url,data).json() # json返回值  .text  7200秒（2小时）有效,不能频繁调 频率拦截 缓存
print(res['access_token'])  

AttributeError: 'dict' object has no attribute 'access_token'

In [3]:
print(res['access_token'])  

hP0CDQj0cGf4SPBY9HO2bbOnPZKk8zQH2GzaX_EpoXy2NBlHPVM0wrubqa-5MbmNizbbVZUAsO4S6DqhxzGUwklU6-wFKyV-NMF0Kbxx8BAuuvkl7Bfh47HrlbsS-pBLyQkYlcxuOrTcLcpLL3NNTp5FvDirn7wcxdcMebIqREytYpuKADkjdkYvD15NR6Y-Iyu1aLwjcvW80lbRHxRrWQ


In [4]:
import requests 

# 企业微信 wechat weixin
# 弓长投资
# 15157186426
# 张燕华
# 微信企业ID :  ww9ca2ccd8f391f581 
# AgentId : 1000002
# Secret : lbjGCu1aNPJafrahotkvOrqrL-m7NfYehPisHMnthro


url='https://qyapi.weixin.qq.com/cgi-bin/gettoken?'
data = {"corpid":"ww9ca2ccd8f391f581", "corpsecret":"lbjGCu1aNPJafrahotkvOrqrL-m7NfYehPisHMnthro"}

# res = requests.get(url,data).json() # json返回值  .text  7200秒（2小时）有效,不能频繁调 频率拦截 缓存
# print(res['access_token'])  

access_token = "hP0CDQj0cGf4SPBY9HO2bbOnPZKk8zQH2GzaX_EpoXy2NBlHPVM0wrubqa-5MbmNizbbVZUAsO4S6DqhxzGUwklU6-wFKyV-NMF0Kbxx8BAuuvkl7Bfh47HrlbsS-pBLyQkYlcxuOrTcLcpLL3NNTp5FvDirn7wcxdcMebIqREytYpuKADkjdkYvD15NR6Y-Iyu1aLwjcvW80lbRHxRrWQ"

url='https://qyapi.weixin.qq.com/cgi-bin/message/send?debug=1&access_token=' + access_token
data = {
        "touser" : "@all",   # 成员ID列表 "UserID1|UserID2|UserID3",
        # "toparty" : "PartyID1|PartyID2", # 部门ID列表 @all忽略本参数
        # "totag" : "TagID1 | TagID2", # 标签ID列表 @all忽略本参数
        "msgtype" : "text",  # 消息类型text
        "agentid" : 1000002,  # 企业应用的id 整型
        "text" : { "content" : 'Hello 2022 Hwr!' },
        # "safe":1,  # 保密消息
        # "enable_id_trans": 1, # 是否开启id转译
        # "enable_duplicate_check": 1,  # 是否开启重复消息检查
        # "duplicate_check_interval": 1800  # 是否重复消息检查的时间间隔，默认1800s，最大不超过4小时
    }

res = requests.post(url,json=data).json()
print(res)



{'errcode': 0, 'errmsg': 'ok', 'msgid': 'mrVtVXE39it1tWVvd57npC59J-4uRFiUWzv4d3O6enxJ777ZqhbNVWR3dIiM7dZvlb8tKP0rQpPS2SrdPjVHhQ'}
